## Variaciones sobre el modelo inicial 

Vamos a implementar alguna modificación sobre el modelo inicial propuesto en la primera sesión.

Para empezar vamos a definir un conjunto de validación. Con este conjunto de calidación extraído del propio conjunto de entrenamiento estimaremos los mejores parámetros. Dejaremos el conjunto de test sólo para estimar el acierto final con estos mejores parámetros.

Primero leemos los datos y normalizamos:

In [1]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (60000, 28, 28)
test set (10000, 28, 28)


## Partición training/validación

Nos quedaremos con un 80% de los datos para entrenar y un 20% para validar:

In [2]:
from sklearn.model_selection import train_test_split

x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train2.shape)
print('val set', x_val.shape)

training set (48000, 784)
val set (12000, 784)


## Experimentación con conjunto de validación

Cualquier parámetro a modificar en nuestro modelo debería ser probado sobre el conjunto de validación y escoger aquel que produjera el mejor resultado para probar ese (y sólo ese) sobre el conjunto de test.

In [8]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=5  

hdim=[128,256,512,1024]
best_acc=0.0
for h in hdim:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=0.01, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train2, y_train2,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) ## <--- OJO validation set
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
model.add(Dense(besth, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))  




Epoch 1/5
375/375 [==============================] - 1s 2ms/step - loss: 0.5435 - accuracy: 0.8495 - val_loss: 0.3122 - val_accuracy: 0.9116
Epoch 2/5
375/375 [==============================] - 1s 2ms/step - loss: 0.2821 - accuracy: 0.9189 - val_loss: 0.2495 - val_accuracy: 0.9277
Epoch 3/5
375/375 [==============================] - 1s 2ms/step - loss: 0.2309 - accuracy: 0.9352 - val_loss: 0.2155 - val_accuracy: 0.9392
Epoch 4/5
375/375 [==============================] - 1s 2ms/step - loss: 0.1977 - accuracy: 0.9444 - val_loss: 0.1908 - val_accuracy: 0.9455
Epoch 5/5
375/375 [==============================] - 1s 2ms/step - loss: 0.1729 - accuracy: 0.9506 - val_loss: 0.1732 - val_accuracy: 0.9525
Epoch 1/5
375/375 [==============================] - 1s 2ms/step - loss: 0.5187 - accuracy: 0.8567 - val_loss: 0.3007 - val_accuracy: 0.9156
Epoch 2/5
375/375 [==============================] - 1s 2ms/step - loss: 0.2725 - accuracy: 0.9227 - val_loss: 0.2439 - val_accuracy: 0.9317
Epoch 3/5
375

## EARLY STOPPING

De la multitud de parámetros a decidir en la definición y entrenamiento de las redes neuronales hay uno que podríamos automatizar. Es el número de epochs a emplear. Dado que tenemos un conjunto de validación vamos a emplearlo para monitorizar cómo evoluciona la convergencia. Si en algún momento el descenso por gradiente parece haber alcanzado una zona de meseta sobre dicho conjunto de validación podremos detener el entrenamiento.

In [9]:
model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train2, y_train2,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback])  



Epoch 1/100
375/375 [==============================] - 2s 5ms/step - loss: 0.4935 - accuracy: 0.8659 - val_loss: 0.2941 - val_accuracy: 0.9177
Epoch 2/100
375/375 [==============================] - 2s 5ms/step - loss: 0.2631 - accuracy: 0.9257 - val_loss: 0.2329 - val_accuracy: 0.9367
Epoch 3/100
375/375 [==============================] - 2s 5ms/step - loss: 0.2130 - accuracy: 0.9409 - val_loss: 0.1992 - val_accuracy: 0.9448
Epoch 4/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1793 - accuracy: 0.9502 - val_loss: 0.1731 - val_accuracy: 0.9518
Epoch 5/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1556 - accuracy: 0.9563 - val_loss: 0.1544 - val_accuracy: 0.9571
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1361 - accuracy: 0.9627 - val_loss: 0.1411 - val_accuracy: 0.9618
Epoch 7/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1211 - accuracy: 0.9668 - val_loss: 0.1305 - val_accuracy: 0.9636

## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.025, 0.01]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test.

---

In [15]:
batch_size = 128
for hdim in [512, 1024]:
    for hden in [1,2,3]:
        for l_rate in [0.025, 0.01]:
            model = Sequential()
            model.add(Input(784))
            for i in range(hden):
                model.add(Dense(hdim, activation='relu'))
            model.add(Dense(num_classes, activation='softmax'))

            
            sgd=SGD(learning_rate=l_rate, momentum=0.9)

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

            model.compile(loss='categorical_crossentropy',
                        optimizer=sgd,
                        metrics=['accuracy'])

            epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
            history = model.fit(x_train2, y_train2,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_val, y_val),
                                callbacks=[callback])  
            
            if history.history['val_accuracy'][-1]>best_acc:
                best_acc=history.history['val_accuracy'][-1]
                besthden = hden
                besthdim = hdim
                bestl_rate= l_rate

print("=============================")
print("Best acc", best_acc)
print("Best hden", besthden)
print("Best hdim", besthdim)
print("Best learning rate",bestl_rate)
print("=============================")



Epoch 1/100
375/375 [==============================] - 1s 3ms/step - loss: 0.3842 - accuracy: 0.8898 - val_loss: 0.2282 - val_accuracy: 0.9359
Epoch 2/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1880 - accuracy: 0.9471 - val_loss: 0.1622 - val_accuracy: 0.9553
Epoch 3/100
375/375 [==============================] - 1s 2ms/step - loss: 0.1368 - accuracy: 0.9608 - val_loss: 0.1328 - val_accuracy: 0.9629
Epoch 4/100
375/375 [==============================] - 1s 2ms/step - loss: 0.1077 - accuracy: 0.9700 - val_loss: 0.1163 - val_accuracy: 0.9683
Epoch 5/100
375/375 [==============================] - 1s 2ms/step - loss: 0.0893 - accuracy: 0.9751 - val_loss: 0.1069 - val_accuracy: 0.9701
Epoch 6/100
375/375 [==============================] - 1s 2ms/step - loss: 0.0741 - accuracy: 0.9801 - val_loss: 0.0939 - val_accuracy: 0.9723
Epoch 7/100
375/375 [==============================] - 1s 2ms/step - loss: 0.0636 - accuracy: 0.9825 - val_loss: 0.0907 - val_accuracy: 0.9722

In [16]:
print(best_acc)
print(besthden)
print(besthdim)
print(bestl_rate)

0.9826666712760925
3
1024
0.025


In [17]:

## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
for i in range(besthden):
    model.add(Dense(besthdim, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=bestl_rate, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación
history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback])  
    


Epoch 1/100
469/469 [==============================] - 6s 11ms/step - loss: 0.3003 - accuracy: 0.9133 - val_loss: 0.1304 - val_accuracy: 0.9595
Epoch 2/100
469/469 [==============================] - 6s 12ms/step - loss: 0.1015 - accuracy: 0.9697 - val_loss: 0.1025 - val_accuracy: 0.9697
Epoch 3/100
469/469 [==============================] - 7s 14ms/step - loss: 0.0649 - accuracy: 0.9805 - val_loss: 0.0735 - val_accuracy: 0.9777
Epoch 4/100
469/469 [==============================] - 7s 14ms/step - loss: 0.0443 - accuracy: 0.9862 - val_loss: 0.0655 - val_accuracy: 0.9796
Epoch 5/100
469/469 [==============================] - 7s 15ms/step - loss: 0.0313 - accuracy: 0.9903 - val_loss: 0.0802 - val_accuracy: 0.9770
Epoch 6/100
469/469 [==============================] - 7s 15ms/step - loss: 0.0209 - accuracy: 0.9940 - val_loss: 0.0651 - val_accuracy: 0.9815


In [18]:
history.history['val_accuracy'][-1]

0.9815000295639038

## Model checkpoint

Los model checkpoint permiten almacenar el estado del modelo en un punto intermedio del entrenamiento. Esto es útil para poder recuperar el modelo en caso de que el entrenamiento se interrumpa por cualquier motivo. Así como para poder recuperar el mejor modelo obtenido durante el entrenamiento.


In [19]:
batch_size=128
epochs=100 

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu')) 
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) 

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.fit(x_train2, y_train2,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback, checkpoint]) ## <--- dos callbacks, el early_stopping y el model_checkpoint


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/100
363/375 [============================>.] - ETA: 0s - loss: 0.5034 - accuracy: 0.8629
Epoch 1: val_accuracy improved from -inf to 0.91900, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.4963 - accuracy: 0.8646 - val_loss: 0.2893 - val_accuracy: 0.9190
Epoch 2/100
 41/375 [==>...........................] - ETA: 1s - loss: 0.2824 - accuracy: 0.9204

/home/pegi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


362/375 [===========================>..] - ETA: 0s - loss: 0.2622 - accuracy: 0.9267
Epoch 2: val_accuracy improved from 0.91900 to 0.93525, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.2612 - accuracy: 0.9272 - val_loss: 0.2343 - val_accuracy: 0.9352
Epoch 3/100
368/375 [============================>.] - ETA: 0s - loss: 0.2123 - accuracy: 0.9409
Epoch 3: val_accuracy improved from 0.93525 to 0.94625, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.2120 - accuracy: 0.9411 - val_loss: 0.1953 - val_accuracy: 0.9463
Epoch 4/100
375/375 [==============================] - ETA: 0s - loss: 0.1778 - accuracy: 0.9505
Epoch 4: val_accuracy improved from 0.94625 to 0.95233, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.1778 - accuracy: 0.9505 - val_loss: 0.1743 - val_accuracy: 0.9523
Epoch 5/100
374/375 [============================>.] - ETA: 0s - loss:

## Ajustar parámetros

Es posible que en el ejemplo anterior el entrenamiento haya acabado prematuramente por el efecto del Early Stopping. En ese caso, el modelo que se ha obtenido no es el mejor posible. Para obtener el mejor modelo quizás sea necesario aumentar el número de epochs y para ello modificar algún parámetro del early stopping.

## Ejercicio: 

Modificar el early stopping y emplear un modelo con los mejores parámetros (número de capas ocultas, learning rate, batch size, etc) para evaluar finalmente el modelo guardado sobre el test.


In [7]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

best_acc = 0.9826666712760925
besthden = 3
besthdim = 1024
bestl_rate = 0.025

batch_size=128
epochs=100 
for min_d in [0.01,0.001,0.0001,0.00001]:
    model = Sequential()

    model.add(Input(784))
    for _ in range(besthden):
        model.add(Dense(besthdim, activation='relu')) 
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=bestl_rate, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
                optimizer=sgd,
                metrics=['accuracy'])
        
    

    callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=min_d , patience=3) 

    checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

    model.fit(x_train2, y_train2,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks=[callback, checkpoint]) ## <--- dos callbacks, el early_stopping y el model_checkpoint


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/100
374/375 [============================>.] - ETA: 0s - loss: 0.3432 - accuracy: 0.9018
Epoch 1: val_accuracy improved from -inf to 0.95792, saving model to best_model.h5
375/375 [==============================] - 5s 12ms/step - loss: 0.3426 - accuracy: 0.9020 - val_loss: 0.1449 - val_accuracy: 0.9579
Epoch 2/100
373/375 [============================>.] - ETA: 0s - loss: 0.1141 - accuracy: 0.9653
Epoch 2: val_accuracy improved from 0.95792 to 0.97067, saving model to best_model.h5
375/375 [==============================] - 4s 12ms/step - loss: 0.1141 - accuracy: 0.9653 - val_loss: 0.1001 - val_accuracy: 0.9707
Epoch 3/100
372/375 [============================>.] - ETA: 0s - loss: 0.0744 - accuracy: 0.9781
Epoch 3: val_accuracy improved from 0.97067 to 0.97308, saving model to best_model.h5
375/375 [==============================] - 5s 12ms/step - loss: 0.0742 - accuracy: 0.9781 - val_loss: 0.0849 - val_accuracy: 0.9731
Epoch 4/100
371/375 [============================>.] - ETA